# **Processing & Analyzing the Data**

First we take a look of the structure of the dataset with the objective of seek which variable we have to analyze and create:

In [16]:
SELECT TOP 10
    *
FROM [202103-divvy-tripdata]

(10 rows affected)

Total execution time: 00:00:00.028

ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0000465CF790D824,electric_bike,2021-03-08 18:08:23.0000000,2021-03-08 19:34:29.0000000,NULL,NULL,NULL,NULL,4192,-8772,4193,-8767,casual
00004F0F77E12E18,classic_bike,2021-03-24 17:08:00.0000000,2021-03-24 18:20:07.0000000,Calumet Ave & 18th St,13102,Michigan Ave & 18th St,13150,41857612,-87619408,41857812,-8762455,casual
000081CDC629355A,classic_bike,2021-03-08 17:48:21.0000000,2021-03-08 18:01:40.0000000,Ashland Ave & Grand Ave,13434,Ashland Ave & Grand Ave,13434,41891072,-87666608,41891072,-87666608,member
0000D6609083F8C8,docked_bike,2021-03-13 00:48:47.0000000,2021-03-13 01:51:12.0000000,Rush St & Hubbard St,KA1503000044,Rush St & Hubbard St,KA1503000044,41890172,-87626184,41890172,-87626184,casual
0000E746AFFDE10D,classic_bike,2021-03-07 15:56:11.0000000,2021-03-07 16:14:26.0000000,Leavitt St & Division St,658,Clinton St & Lake St,13021,41902996,-87683824,41885636,-87641824,member
00013A098E40CBAD,classic_bike,2021-03-09 16:59:32.0000000,2021-03-09 17:11:37.0000000,Theater on the Lake,TA1308000001,Broadway & Barry Ave,13137,41926276,-87630832,4193758270717952,-876440958533632,casual
00013FFF86FF03DA,classic_bike,2021-03-08 16:35:49.0000000,2021-03-08 17:16:21.0000000,Lake Shore Dr & Monroe St,13300,Lake Shore Dr & Monroe St,13300,41880960,-87616744,41880960,-87616744,casual
0001BC6649540A9E,electric_bike,2021-03-09 09:00:54.0000000,2021-03-09 09:06:04.0000000,Ritchie Ct & Banks St,KA1504000134,Clybourn Ave & Division St,TA1307000115,41906936504909820,-8762620731457536,41904617222569980,-8764043976245248,member
0001DA2D84B47067,classic_bike,2021-03-11 17:10:19.0000000,2021-03-11 17:42:06.0000000,St. Clair St & Erie St,13016,Clark St & Grace St,TA1307000127,4189434580828160,-8762279818428416,4195078,-87659168,member
00020B17897E5C2E,classic_bike,2021-03-09 06:58:19.0000000,2021-03-09 07:18:56.0000000,Clark St & Grace St,TA1307000127,Sheffield Ave & Waveland Ave,TA1307000126,4195078,-87659168,41949400,-87654528,member


So, after having analyzed our dataset structure, we make some calculations in order to get some inights about differences found between types of users (**member\_casual**). I am going to include some aggregate functions like **COUNT, SUM** and **AVG:**

In [1]:
--Numero de viajes, Promedio de viajes y tiempo total de viajes, por tipo de usuario (member or casual)

SELECT  
	member_casual,
	COUNT(ride_length) AS number_of_trips, 
	AVG(ride_length) AS average_ride, 
	SUM(ride_length) AS total_time_trips

FROM (
	SELECT	
		member_casual, 
		DATEDIFF(SECOND,started_at, ended_at) AS ride_length 
	FROM [202103-divvy-tripdata]
) AS temp_table
GROUP BY member_casual

(2 rows affected)

Total execution time: 00:00:00.044

member_casual,number_of_trips,average_ride,total_time_trips
member,144463,838,121088499
casual,84033,2289,192395561


The _casual riders_ have less **number\_of\_trips** but a longer **average\_ride** time (in minutes) than  _member riders_. Because of that, the sum of **total\_time\_trips** is greater in _casual riders_. We have to explore why or how this happen.

Then we search a relationship between the **day\_of\_week** the user started the trip, by type of user (**member\_casual**) in order to indentify some differences. But first, I look up for the mode of **date\_of\_week** in general terms:

In [15]:
SELECT  
	day_of_week,
	COUNT(day_of_week) AS number_trips
FROM (
	SELECT	
		member_casual, 
		rideable_type,
		DATEPART(DW, started_at) AS day_of_week  
	FROM [202103-divvy-tripdata]
) AS temp_table1
GROUP BY day_of_week
ORDER BY number_trips DESC

(7 rows affected)

Total execution time: 00:00:00.059

day_of_week,number_trips
7,45252
1,35627
2,34825
3,34000
4,31669
6,25657
5,21466


As you can see, there is an important preference for 7 (Sunday), following by 1 (Monday) in which **day\_of\_week** the user starts its ride (_Remember this: Monday=1 and Sunday=7)._
Now we try to analyze the preference of **day\_of\_week** separated by type of user (**member\_casual**) in order to keep trying find differences. Also we aggregate functions like **COUNT, AVG** and **SUM,** and group them by type of user(**member\_casual**) and **day\_of\_week** in order to get further detail about our analysis.

In [4]:

SELECT  
	member_casual,
	day_of_week,
	COUNT(ride_length) AS number_of_trips, 
	AVG(ride_length) AS average_ride, 
	SUM(ride_length) AS total_time_trips

FROM (
	SELECT	
		member_casual, 
		rideable_type,
		DATEDIFF(SECOND,started_at, ended_at) AS ride_length,  
		DATEPART(DW, started_at) AS day_of_week	
	FROM [202103-divvy-tripdata]
) AS temp_table
WHERE member_casual='casual'
GROUP BY member_casual, day_of_week
ORDER BY member_casual, number_of_trips DESC
------------------------------------------------------------------

SELECT  
	member_casual,
	day_of_week,
	COUNT(ride_length) AS number_of_trips, 
	AVG(ride_length) AS average_ride, 
	SUM(ride_length) AS total_time_trips

FROM (
	SELECT	
		member_casual, 
		rideable_type,
		DATEDIFF(SECOND,started_at, ended_at) AS ride_length,  
		DATEPART(DW, started_at) AS day_of_week	
	FROM [202103-divvy-tripdata]
) AS temp_table
WHERE member_casual='member'
GROUP BY member_casual, day_of_week
ORDER BY member_casual, number_of_trips DESC

(7 rows affected)

(7 rows affected)

Total execution time: 00:00:00.120

member_casual,day_of_week,number_of_trips,average_ride,total_time_trips
casual,7,22090,2537,56063610
casual,1,17342,2481,43041420
casual,2,11990,2648,31752832
casual,3,10463,2160,22608134
casual,4,8883,1724,15316765
casual,6,7777,1751,13623522
casual,5,5488,1820,9989278


member_casual,day_of_week,number_of_trips,average_ride,total_time_trips
member,3,23537,808,19037584
member,7,23162,952,22060680
member,2,22835,847,19357166
member,4,22786,775,17666738
member,1,18285,969,17721068
member,6,17880,765,13678611
member,5,15978,723,11566652


From the tables shown above, we can conclude the following:

- There is differences of which **day\_of\_week** each type of user prefer take the ride. The **number\_of\_trips** column shows us that _casual riders_ prefer Sunday followed by Monday, and _member riders_ prefer Wednesday followed by Sunday.
- Also, wee can see which **day\_of\_week** the **average\_ride** time (in minutes) is longer. In the case of _casual riders_, the **average\_ride** time is longer in Tuesday (44 minutes), Sunday (42) and Monday (41).
- For _member riders_, the longer **average\_ride** time are shorter than _casual riders:_ Monday (16 minutes), Sunday (15 ) and Tuesday (14). The average times of _member riders_ are more atomized than _casual riders._
- Notice that there is not much correlation between **average\_ride** time of each **day\_of\_week** and the **number\_of\_trips** grouped by **day\_of\_week**.

 So, we explore a little more into data, looking  which the longest trips are (in _Hours_ because of the magnitude of longest ride\_length):

In [10]:
SELECT TOP 50
	member_casual,
	started_at,
	ended_at,
	DATEDIFF(HOUR,started_at,ended_at) AS ride_length,
	DATEPART(DW, started_at) AS day_of_week,    -- DAY OF THE WEEK (1 TO 7)
	rideable_type

FROM [202103-divvy-tripdata]
ORDER BY ride_length desc


(50 rows affected)

Total execution time: 00:00:00.080

member_casual,started_at,ended_at,ride_length,day_of_week,rideable_type
casual,2021-03-08 16:48:24.0000000,2021-03-30 16:50:03.0000000,528,2,docked_bike
casual,2021-03-19 14:11:08.0000000,2021-04-02 12:45:14.0000000,334,6,docked_bike
casual,2021-03-02 17:48:41.0000000,2021-03-15 09:39:25.0000000,304,3,docked_bike
casual,2021-03-06 22:37:58.0000000,2021-03-18 22:08:43.0000000,288,7,docked_bike
casual,2021-03-06 22:58:09.0000000,2021-03-18 22:09:49.0000000,288,7,docked_bike
casual,2021-03-22 13:22:48.0000000,2021-04-02 14:27:23.0000000,265,2,docked_bike
casual,2021-03-20 17:35:12.0000000,2021-03-30 13:56:23.0000000,236,7,docked_bike
casual,2021-03-18 21:23:58.0000000,2021-03-27 22:05:30.0000000,217,5,docked_bike
casual,2021-03-20 23:58:11.0000000,2021-03-29 17:01:59.0000000,210,7,docked_bike
casual,2021-03-21 18:20:50.0000000,2021-03-29 21:30:13.0000000,195,1,docked_bike


The results show an anomaly in the r**ide\_length** of the longest rides. Some insights we capture:

- The duration of the longest ride is 528 hours, which is too much in my opinion. 
- Then the second one is 334 hours, and so on until completing the top 100 with 25 hours. 
- All the three-digits duration rides are which **rideable\_type** are _'docked\_bikes'_ and the type of rider (**member\_casual**) is _casual ._
- We also found that these very long rides are mainly from _casual riders_. 
- The longest ride from _member riders_ is 26 hours (row n°80).

Now we take a deeper look at  **rideable\_types** separated by types of users (**member\_casual**):

In [3]:
SELECT  
	member_casual,
	rideable_type,
	COUNT(ride_length) AS number_of_trips, 
	AVG(ride_length) AS average_ride, 
	SUM(ride_length) AS total_time_trips

FROM (
	SELECT	
		member_casual, 
		rideable_type,
		DATEDIFF(MINUTE,started_at, ended_at) AS ride_length,  --CREAR UNA TABLA TABLA CON TODOS LOS CALCULOS 
		DATEPART(DW, started_at) AS day_of_week	
	FROM [202103-divvy-tripdata]
) AS temp_table
WHERE member_casual='casual'
GROUP BY member_casual, rideable_type
ORDER BY member_casual, number_of_trips DESC


SELECT  
	member_casual,
	rideable_type,
	COUNT(ride_length) AS number_of_trips, 
	AVG(ride_length) AS average_ride, 
	SUM(ride_length) AS total_time_trips

FROM (
	SELECT	
		member_casual, 
		rideable_type,
		DATEDIFF(MINUTE,started_at, ended_at) AS ride_length,  --CREAR UNA TABLA TABLA CON TODOS LOS CALCULOS 
		DATEPART(DW, started_at) AS day_of_week	
	FROM [202103-divvy-tripdata]
) AS temp_table
WHERE member_casual='member'
GROUP BY member_casual, rideable_type
ORDER BY member_casual, number_of_trips DESC


(3 rows affected)

(2 rows affected)

Total execution time: 00:00:00.453

member_casual,rideable_type,number_of_trips,average_ride,total_time_trips
casual,classic_bike,45528,31,1435951
casual,electric_bike,22848,21,492410
casual,docked_bike,15657,81,1278390


member_casual,rideable_type,number_of_trips,average_ride,total_time_trips
member,classic_bike,107017,14,1519896
member,electric_bike,37446,13,498135


We can see that docked\_bike rides  have an **average\_ride** time of 81 minutes wich explain what we mentioned above. It would be useful know which are the trips that is most repeated (or Mode), grouped by **ride\_legth** time. For that we have to round the ride\_length variable removing decimals to simplify the results table:

In [8]:
SELECT  TOP 50
	ride_length,
	COUNT(ride_length) AS number_of_trips, 
	AVG(ride_length) AS average_ride, 
	SUM(ride_length) AS total_time_trips

FROM (
	SELECT	
		ROUND(DATEDIFF(MINUTE,started_at, ended_at),0) AS ride_length,  -- USE ROUND FUNCTIONS TO REMOVE DECIMALS
		DATEPART(DW, started_at) AS day_of_week	
	FROM [202103-divvy-tripdata]
) AS temp_table
GROUP BY ride_length
ORDER BY number_of_trips DESC

(50 rows affected)

Total execution time: 00:00:00.054

ride_length,number_of_trips,average_ride,total_time_trips
6,12578,6,75468
7,12442,7,87094
5,12337,5,61685
8,11821,8,94568
4,11168,4,44672
9,10857,9,97713
10,10043,10,100430
11,9116,11,100276
12,8531,12,102372
3,8199,3,24597


The mode of the variable **ride\_length** is 6 minutes followed by 7, 5, 8, 4, 9 minutes and so on (rounding the numbers to whole number closer, removing decimals). As you noticed, the **ride\_length** and **average\_ride** are equals because what we explained. Fo r example: the **average\_ride** time grouped by **ride\_length** of 6 minutes, is calculated by numbers between 5.5 and 6.4 (5.5,6.5\], so it is logical that the result tends towards the nearest whole number, in this case 6 minutes.

**<u>Removing Outliers</u>**

Now we try to find which cells we have to remove in order to achieve statistical significance. At 95% of confidence level we need z-core values lower than -1.96 or higher than 1.96. For 99% of confidence level we include the values with z-score lower than -2.96 and higher than 2.96.

In [14]:
WITH trips1 AS (
	SELECT ride_id,
	member_casual,
	ROUND(DATEDIFF(MINUTE, started_at, ended_at),0) AS ride_length
	FROM [202103-divvy-tripdata]
	),
	rides_stats AS (
	SELECT
		AVG(ride_length) AS ride_average,
		STDEV(ride_length)	AS stan_dev
	FROM (SELECT member_casual, DATEDIFF(MINUTE,started_at, ended_at) AS ride_length FROM [202103-divvy-tripdata]) AS temp_table
	)
SELECT TOP 500
	ride_id,
	ride_length,
	member_casual,
	ABS(trips1.ride_length - rides_stats.ride_average)/rides_stats.stan_dev AS z_score_ride
FROM trips1, rides_stats
ORDER BY z_score_ride DESC

(500 rows affected)

Total execution time: 00:00:00.168

ride_id,ride_length,member_casual,z_score_ride
271D9E93A68C9A17,31682,casual,"205,02022110947186"
9F66323DE1CE4C08,20074,casual,"129,8504571600483"
F173092104F0C6AA,18231,casual,"117,91576772528026"
5752EBD1DB94ACB3,17251,casual,"111,56959537255499"
EBD8B407D4DE1469,17231,casual,"111,4400816510708"
61486C9900757ABE,15905,casual,"102,85332191666903"
7D3DF23CE224F140,14181,casual,"91,68923912473191"
79A5C8E0CB0D6C52,13002,casual,"84,05440524323895"
83F37001BA988B9F,12543,casual,"81,0820653351768"
A8A6F0B128472D2C,11710,casual,"75,6878188353603"


The results showed that values with **ride\_length** time greater than 480 minutes (row n°433) have a z-score value higher than 2.96 (absolute value). This give us a confidence level of 95%. 

If we exclude all theese data from the dataset, we will see how the average

In [18]:
--AISLANDO LOS OUTLIERS (las docked_bikes arrojaban valores ride_length muy elevados)

SELECT  
	member_casual,
	COUNT(ride_length) AS number_of_trips, 
	AVG(ride_length) AS average_ride, 
	SUM(ride_length) AS total_time_trips

FROM (
	SELECT	
		member_casual, 
		DATEDIFF(MINUTE,started_at, ended_at) AS ride_length  --CREAR UNA TABLA TABLA CON TODOS LOS CALCULOS 
	FROM [202103-divvy-tripdata]
) AS temp_table
GROUP BY member_casual

-- REMOVING OUTLIERS > 430 MINUTES
SELECT  
	member_casual,
	COUNT(ride_length) AS number_of_trips, 
	AVG(ride_length) AS average_ride, 
	SUM(ride_length) AS total_time_trips

FROM (
	SELECT	
		member_casual, 
		DATEDIFF(MINUTE,started_at, ended_at) AS ride_length  --CREAR UNA TABLA TABLA CON TODOS LOS CALCULOS 
	FROM [202103-divvy-tripdata]
) AS temp_table
WHERE ride_length < 480 
GROUP BY member_casual      

(2 rows affected)

(2 rows affected)

Total execution time: 00:00:00.112

member_casual,number_of_trips,average_ride,total_time_trips
casual,84033,38,3206751
member,144463,13,2018031


member_casual,number_of_trips,average_ride,total_time_trips
casual,83675,29,2438181
member,144415,13,1967721
